In [6]:
from modules.browser_control import BrowserControl
from facebook_register import FacebookRegister

bc = BrowserControl()
fr = FacebookRegister()
print(fr.browser_id)

2024/07/29 16:48:03 INFO     获取手机号码[16756951851]-->16756951851,雅玲,張,1999,4,27,2EEBU4guPw6PTvr

                    INFO     create browser:44183e644d464d2f9ac54be334538d65

2024/07/29 16:48:26 INFO     connect driver:{'success': True, 'data': {'ws':                                       
                             'ws://127.0.0.1:63753/devtools/browser/c459d263-8287-40df-b4d6-898384a091c5', 'http': 
                             '127.0.0.1:63753', 'coreVersion': '126', 'driver':                                    
                             'C:\\Users\\lqj\\AppData\\Roaming\\BitBrowser\\chromedriver\\126\\chromedriver.exe',  
                             'seq': 100, 'name': 'facebook-16756951851', 'remark':                                 
                             '16756951851,雅玲,張,1999,4,27,2EEBU4guPw6PTvr', 'pid': 7044}}

44183e644d464d2f9ac54be334538d65


In [7]:
fr.driver.get('https://mbasic.facebook.com')

In [9]:

from selenium.webdriver.common.by import By
helper = fr.helper
sign_up_btn = helper.find_or_fail(By.NAME,'sign_up')
sign_up_btn.click()


In [10]:
username_input = helper.find_or_fail(By.NAME,'reg_email__')
lastname_input = helper.find_or_fail(By.NAME,'lastname')
firstname_input = helper.find_or_fail(By.NAME,'firstname')
password_input = helper.find_or_fail(By.NAME,'reg_passwd__')
year_choose = helper.find_or_fail(By.NAME,'birthday_year')
month_choose = helper.find_or_fail(By.NAME,'birthday_month')
day_choose = helper.find_or_fail(By.NAME,'birthday_day')
sex_click = helper.find_or_fail(By.NAME,'sex')

username_input.send_keys(f'8616756951851')
lastname_input.send_keys(f'{fr.lastname}')
firstname_input.send_keys(f'{fr.firstname}')
password_input.send_keys(f'{fr.fb_password}')
year_choose.send_keys(f'{fr.year}')
month_choose.send_keys(f'{fr.month}')
day_choose.send_keys(f'{fr.day}')
sex_click.click()

In [ ]:
fr.sms.get_message()